# Interpreting our model 

In this notebook we demonstrate how to interpret our model using  `Captum` library. 

We show how to use interpretation hooks to examine and better understand embeddings, sub-embeddings, and attention layers. 

Note: Before running this tutorial, please use `pip install -r requirements.txt` to install the required python packages.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from src.config import Config
from src.models import SaintEncoderDecoderModel

from transformers import AutoTokenizer, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# settings
config_path = "config/saint.yaml"
config = Config.load(config_path)

# configuring saint model 
encoder_config = BertConfig(**config.encoder_config)
decoder_config = BertConfig(**config.decoder_config)

encoder_config.max_position_embeddings = config.window_size
decoder_config.max_position_embeddings = config.window_size

# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = './output/saint_francois/model.pt'

model = SaintEncoderDecoderModel(encoder_config, decoder_config)
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")))
model.to(device)

# use if cuda enabled on device
# model.to(config.device)

model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# tokenizer = AutoTokenizer.from_pretrained(model_path)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# NOTES ON SAINT ARCHITECTURE 
# Encoder takes sequence of exercise embeddings as queries, keys and values; produces output through repeated self-attention mechanism
# Decoder takes sequence of response embeddings as queries, keys and values; then alternately applies self-attention and attention layers to encoder output

# overall SAINT
# input - sequences of exercise info and response info
# ouput - kth user response r_k

A helper function to perform forward pass of the model and make predictions.

In [7]:
# Below is from original BERT SQUAD ex, needs replacing
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits

Defining a custom forward function that will allow us to access the start and end postitions of our prediction using `position` input argument.

In [ ]:
Need to define a custom forward function that allows you to access the bounds of our prediction